In [13]:
# python script to read stock info from csv file

import pandas as pd;
import os as os;

path = '../data/stock_market_dataset/stocks'

files = os.listdir(path)

for f in files:
    fpath = '../data/stock_market_dataset/stocks/'+ f
    data = pd.read_csv(fpath)
    print(f, data.shape)
    



RIV.csv (1065, 7)
ANTE.csv (3121, 7)
CSCO.csv (7589, 7)
PRI.csv (2518, 7)
NZF.csv (4656, 7)
HLNE.csv (778, 7)
UNT.csv (10336, 7)
HUBS.csv (1379, 7)
GPL.csv (3484, 7)
SBGI.csv (6249, 7)
UFCS.csv (10097, 7)
TEAF.csv (257, 7)
RJZ.csv (3135, 7)
GFED.csv (6289, 7)
AIRT.csv (9060, 7)
SITC.csv (6841, 7)
NXN.csv (6998, 7)
BNY.csv (4699, 7)
RHE.csv (813, 7)
ISSC.csv (4945, 7)
KNDI.csv (3167, 7)
EFT.csv (3950, 7)
TRNS.csv (10674, 7)
CARO.csv (1482, 7)
NTZ.csv (6771, 7)
AEIS.csv (6134, 7)
UAL.csv (3563, 7)
TROW.csv (8572, 7)
ISRG.csv (4979, 7)
EGP.csv (11884, 7)
EFC.csv (2386, 7)
EEX.csv (737, 7)
NVR.csv (8746, 7)
PLYM.csv (708, 7)
TELA.csv (99, 7)
FEIM.csv (10033, 7)
WOR.csv (10100, 7)
TLRY.csv (429, 7)
PRGO.csv (7126, 7)
LECO.csv (6543, 7)
ALTG.csv (110, 7)
RPT.csv (7877, 7)
VCYT.csv (1616, 7)
USAC.csv (1816, 7)
COHN.csv (4039, 7)
NBW.csv (4391, 7)
PLOW.csv (2495, 7)
PIC.csv (1739, 7)
PRGX.csv (6045, 7)
ERF.csv (4871, 7)
IFN.csv (6578, 7)
FIHD.csv (1034, 7)
PHG.csv (10098, 7)
UVE.csv (4235, 7)


KeyboardInterrupt: 